## RAGAS Evaluation


documentation: https://docs.ragas.io/en/stable/getstarted/index.html

#### Standard way

it gives Attribute Error LlamaForCausalLM' object has no attribute 'set_run_config'  during evaluation

Tested also using LangchainLLMWrapper. No good results

In [1]:
"""
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings

# embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #they used "BAAI/bge-m3"

# evaluator
model_id = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
"""


'\nfrom transformers import AutoTokenizer, AutoModelForCausalLM\nfrom langchain_community.llms import HuggingFacePipeline\nfrom langchain_community.embeddings import HuggingFaceEmbeddings\n\n# embedding model\nembedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #they used "BAAI/bge-m3"\n\n# evaluator\nmodel_id = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel = AutoModelForCausalLM.from_pretrained(model_id)\n'

In [2]:
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
                    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

In [5]:
from datasets import Dataset 

dataset = Dataset.from_dict(data_samples)

In [9]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ]

#metrics = [
 #   BLEU(), ROUGE(), METEOR(), BertScore(), NIST(), CIDEr(), TER(), Spice() ]

In [10]:
from ragas import evaluate
score = evaluate(dataset, metrics=metrics, llm=model, embeddings=embedding_model)

AttributeError: 'LlamaForCausalLM' object has no attribute 'set_run_config'

In [ ]:
score

### Improving standard way using LangchainLLMWrapper and LangchainEmbeddingsWrapper


In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset


model_name = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.1, 
    repetition_penalty=1.1
)


langchain_llm = HuggingFacePipeline(pipeline=pipe)

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Wrap the LLM and embeddings with the RAGAS wrappers
langchain_llm = LangchainLLMWrapper(langchain_llm)
langchain_embeddings = LangchainEmbeddingsWrapper(embedding_model)


data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}


dataset = Dataset.from_dict(data_samples)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/leobaro/workspace/labs/eurocc/ecc-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# Run evaluation
result = evaluate(
    dataset=dataset,
    llm=langchain_llm,
    embeddings=langchain_embeddings,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    raise_exceptions=False, 
)

# Convert the evaluation result to a pandas DataFrame and print it
evaluation_df = result.to_pandas()
print("Evaluation Results:\n", evaluation_df)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/leobaro/workspace/labs/eurocc/ecc-env/lib/python3.10/site-packages/ragas/executor.py", line 87, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/usr/lib/python3.10/asyncio/base_events.py", line 625, in run_until_complete
    self._check_running()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 584, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
/usr/lib/python3.10/threading.py:1018: RuntimeWarning: coroutine 'Runner._aresults' was never awaited
  self._invoke_excepthook(self)


ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

/home/leobaro/workspace/labs/eurocc/ecc-env/lib/python3.10/site-packages/ipykernel/iostream.py:133: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  for thread, socket in list(self._event_pipes.items()):
/home/leobaro/workspace/labs/eurocc/ecc-env/lib/python3.10/site-packages/ipykernel/iostream.py:123: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  await self._event_pipe_gc()


In [2]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset

# Define the LLM model and tokenizer
#model_name = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
model_name = "sapienzanlp/Minerva-3B-base-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create the pipeline
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.1, 
    repetition_penalty=1.1
)


# Create the HuggingFacePipeline for LangChain
langchain_llm = HuggingFacePipeline(pipeline=pipe)

# Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Wrap the LLM and embeddings with the RAGAS wrappers
langchain_llm = LangchainLLMWrapper(langchain_llm)
langchain_embeddings = LangchainEmbeddingsWrapper(embedding_model)

# Example data samples
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

# Create a dataset from the data samples
dataset = Dataset.from_dict(data_samples)


# Run evaluation
result = evaluate(
    dataset=dataset,
    llm=langchain_llm,
    embeddings=langchain_embeddings,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    raise_exceptions=True, 
    is_async = True  # Ensure exceptions are raised directly
)

# Convert the evaluation result to a pandas DataFrame and print it
evaluation_df = result.to_pandas()
print("Evaluation Results:\n", evaluation_df)


/home/leobaro/workspace/labs/eurocc/ecc-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/sapienzanlp/Minerva-3B-base-v1.0.
403 Client Error. (Request ID: Root=1-669a2c2b-6ed4ea0a216b8d9904acd2c1;64b67d0d-8e8d-47aa-ae83-0d2763a595e6)

Cannot access gated repo for url https://huggingface.co/sapienzanlp/Minerva-3B-base-v1.0/resolve/main/config.json.
Access to model sapienzanlp/Minerva-3B-base-v1.0 is restricted and you are not in the authorized list. Visit https://huggingface.co/sapienzanlp/Minerva-3B-base-v1.0 to ask for access.

In [1]:

# Create the HuggingFacePipeline for LangChain
langchain_llm = HuggingFacePipeline(pipeline=pipe)

# Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Wrap the LLM and embeddings with the RAGAS wrappers
langchain_llm = LangchainLLMWrapper(langchain_llm)
langchain_embeddings = LangchainEmbeddingsWrapper(embedding_model)

# Example data samples
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

# Create a dataset from the data samples
dataset = Dataset.from_dict(data_samples)


# Run evaluation
result = evaluate(
    dataset=dataset,
    llm=langchain_llm,
    embeddings=langchain_embeddings,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    raise_exceptions=True, 
    is_async = True  # Ensure exceptions are raised directly
)

# Convert the evaluation result to a pandas DataFrame and print it
evaluation_df = result.to_pandas()
print("Evaluation Results:\n", evaluation_df)


NameError: name 'HuggingFacePipeline' is not defined

In [ ]:
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}


dataset = Dataset.from_dict(data_samples)

In [5]:
import ragas
run_config = ragas.RunConfig(timeout=10, max_retries=1, max_wait=10)

In [14]:
result = evaluate(
    dataset=dataset,
    llm=langchain_llm,
    embeddings=langchain_embeddings,
    metrics=[
        context_precision#,
        #faithfulness#,
        #answer_relevancy#,
        #context_recall,
    ],
    raise_exceptions=False,
    run_config= run_config # Ensure exceptions are raised directly
    #is_async = True  
)


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "c:\app\BaseToolkit\lib\site-packages\ragas\metrics\_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
  File "c:\app\BaseToolkit\lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 88, in async_wrapped
    return await fn(*args, **kwargs)
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 47, in __call__
    do = self.iter(retry_state=retry_state)
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\__init__.py", line 314, in iter
    return fut.result()
  File "c:\app\BaseToolkit\lib\concurrent\futures\_base.py", line 439, in result
    return self.__get_result()
  File "c:\app\Bas

In [15]:
result

{'context_precision': nan}

#### Ollama case (no Lllamantino)


solved!! it gives RuntimeError: This event loop is already running
to be fixed: TimeoutError

In [6]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [7]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

In [8]:
langchain_embeddings = OllamaEmbeddings(model = "llama3")
langchain_llm = ChatOllama(model = "llama3")

In [16]:
from datasets import Dataset 
data_samples = {
    'question': ['When was the first super bowl?'],#, 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967'],# 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,']], 
    #['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967']#, 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

In [11]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ]

In [12]:
from ragas import evaluate

In [20]:
import asyncio
from ragas.llms.base import RunConfig
result = await asyncio.wait_for(
                    evaluate(dataset, metrics=metrics, llm = langchain_llm, embeddings = langchain_embeddings, run_config=RunConfig(thread_timeout=60)),
                    timeout=600  # 10 minutes timeout 
                    )

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [01:00<?, ?it/s]
Exception in thread Thread-37:
Traceback (most recent call last):
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 258, in __step
    result = coro.throw(exc)
  File "c:\app\BaseToolkit\lib\site-packages\ragas\metrics\_context_recall.py", line 169, in _ascore
    results = await self.llm.generate(
  File "c:\app\BaseToolkit\lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 88, in async_wrapped
    return await fn(*args, **kwargs)
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 57, in __call__
    await self.sleep(do)
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 652, in sleep
    return await future
  File "c:\app\BaseToolkit\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\app\BaseToolkit\lib\asyncio\task

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [13]:
result = evaluate(dataset, metrics=metrics, llm = langchain_llm, embeddings = langchain_embeddings)

Evaluating:   0%|          | 0/8 [01:19<?, ?it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 258, in __step
    result = coro.throw(exc)
  File "c:\app\BaseToolkit\lib\site-packages\ragas\metrics\_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
  File "c:\app\BaseToolkit\lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 88, in async_wrapped
    return await fn(*args, **kwargs)
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 57, in __call__
    await self.sleep(do)
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 652, in sleep
    return await future
  File "c:\app\BaseToolkit\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\app\BaseToolkit\lib\asyncio\ta

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [17]:
result

NameError: name 'result' is not defined